# Cuantización afín y motor QSim
Este cuaderno comenta y demuestra con **ejemplos mínimos**:
- Ecuaciones de **cuantización afín** (§2) ↔ funciones de `src/quantizer.py`.
- **Acumulación INT32** y **requant** (§3) ↔ `src/qsim_engine.py`.
- **Fusión Conv+ReLU** en cuantizado.


In [3]:
# --- Setup rápido para importar src.* desde notebooks/ ---
import sys
from pathlib import Path

# Sube hasta encontrar la carpeta raíz del repo (la que contiene src/)
ROOT = Path.cwd()
while not (ROOT / "src").exists() and ROOT.parent != ROOT:
    ROOT = ROOT.parent

sys.path.insert(0, str(ROOT))
print("Ruta raíz añadida a sys.path →", ROOT)


Ruta raíz añadida a sys.path → c:\Users\josem\Desktop\EL AÑO\OH\ptq-int8-project


In [4]:
import inspect, textwrap
from src.quantizer import affine_params, quantize, dequantize, fit_weight_qparams, fit_act_qparams
from src.qsim_engine import conv2d_int, linear_int, requantize_int32

def show(fn):
    print(f"--- {fn.__module__}.{fn.__name__} ---")
    print(textwrap.dedent(inspect.getsource(fn)))
    
show(affine_params)
show(quantize)
show(dequantize)


--- src.quantizer.affine_params ---
def affine_params(a, b, num_bits=8, symmetric=False, signed=False):
    qmin, qmax = (-(2**(num_bits-1)), 2**(num_bits-1)-1) if signed else (0, 2**num_bits - 1)
    # asegurar que 0 cae en [a,b]
    a = min(a, 0.0); b = max(b, 0.0)
    if symmetric:
        m = max(abs(a), abs(b)); a, b = -m, m
    # escala y zp
    scale = (b - a) / (qmax - qmin) if b > a else 1.0
    if signed:
        zp = 0
    else:
        zp = round(qmin - a / scale)
        zp = int(np.clip(zp, qmin, qmax))
    return float(scale if scale != 0 else 1.0), int(zp), int(qmin), int(qmax)

--- src.quantizer.quantize ---
def quantize(x, scale, zp, qmin, qmax, dtype=np.int8):
    q = np.round(x / scale + zp)
    q = np.clip(q, qmin, qmax).astype(np.int32)
    return q.astype(dtype)

--- src.quantizer.dequantize ---
def dequantize(q, scale, zp):
    return scale * (q.astype(np.float32) - zp)



# 1) Cuantización afín (Sección 2 de la memoria)

**Ecuaciones**
- $x_q = \mathrm{clip}(\mathrm{round}(x/s) + z,\ q_{\min}, q_{\max})$
- $x \approx s \cdot (x_q - z)$

**Interpretación.** `affine_params` fija $s,z$ para un rango $[a,b]$, con variantes simétrica ($z{=}0$) o asimétrica. `quantize`/`dequantize` implementan exactamente esas dos ecuaciones.


In [5]:
import numpy as np
x = np.array([-1.2, -0.1, 0.0, 0.3, 1.0], dtype=np.float32)
s, z, qmin, qmax = affine_params(a=float(x.min()), b=float(x.max()),
                                 num_bits=8, symmetric=False, signed=False)
xq = quantize(x, s, z, qmin, qmax, dtype=np.uint8)
xr = dequantize(xq, s, z)
print("x   :", x)
print("x_q :", xq)
print("x≈  :", xr)
print("err :", np.abs(x - xr))


x   : [-1.2 -0.1  0.   0.3  1. ]
x_q : [  0 127 139 174 255]
x≈  : [-1.1992157  -0.10352941  0.          0.3019608   1.0007843 ]
err : [0.0007844  0.00352941 0.         0.00196078 0.00078428]


# 2) Recuantización tras la acumulación INT32 (Sección 2)

Salida entera de la capa:
$$
y_q = \mathrm{clip}\!\left(\left\lfloor y_{32}\,\frac{s_x s_w}{s_y} \right\rceil + z_y,\ q_{\min}, q_{\max}\right).
$$


In [6]:
y32 = np.array([-1000, -10, 0, 15, 1000], dtype=np.int32)
s_x, s_w, s_y = 0.05, 0.02, 0.03
z_y, qmin, qmax = 10, 0, 255
yq = requantize_int32(y32, s_x, s_w, s_y, z_y, qmin, qmax, out_dtype=np.uint8)
print("y32:", y32)
print("yq :", yq)


y32: [-1000   -10     0    15  1000]
yq : [ 0 10 10 10 43]


## 3) Convolución INT8→INT32 y fusión ReLU (§2, §3)
`conv2d_int` hace el producto sobre **enteros centrados** `(x_q - z_x)*(w_q - z_w)` y acumula en INT32.
Después `requantize_int32` pasa a la salida cuantizada; la **ReLU** en cuantizado es `max(y_q, z_y)`.

> Ejemplo mínimo con tensores pequeños (sin kernels externos).


In [ ]:
# activación 1x1 con 1 canal y kernel 1x1 -> salida 1 canal
xq = np.array([[[[120]]]], dtype=np.uint8)   # (N=1,C=1,H=1,W=1)
wq = np.array([[[[5]]]], dtype=np.int8)      # (OC=1,IC=1,KH=1,KW=1)
z_x, z_w = 128, 0
b32 = np.array([0], dtype=np.int32)

y32 = conv2d_int(xq, wq, z_x, z_w, b32, stride=1, padding=0)
print("y32:", y32)

# escalas de ejemplo
s_x, s_w, s_y = 0.02, 0.05, 0.04
z_y, qmin, qmax = 10, 0, 255
yq = requantize_int32(y32, s_x, s_w, s_y, z_y, qmin, qmax, out_dtype=np.uint8)
yq_relu = np.maximum(yq, z_y)  # ReLU en cuantizado
print("yq (antes ReLU):", yq, " yq_relu:", yq_relu) 


y32: [[[[-40]]]]
yq (antes ReLU): [[[[9]]]]  yq_relu: [[[[10]]]]


## 4) Per-tensor vs Per-channel
Para pesos `per_channel`, `fit_weight_qparams` da `scale` vectorial por canal de salida y en `requantize_int32` se **difunde** como `(1, OC, 1, 1)` (conv) o `(1, Fout)` (lineal). Esto explica los reshape en §3.
